In [1]:
from misc import loadProperties, loadWEKA

props = loadProperties('submitActionClass.properties')
(data, attr) = loadWEKA('youTubeLocationIDWeka.csv', limit=0)

Adjusting spacy's pipeline

In [2]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")

locations = [x for x in props] # Array of known locations from .properties
location_patterns = list(nlp.pipe(locations))

matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
matcher.add("LOCATION", None, *location_patterns)

# Define the custom component
def location_component(doc):
    # Apply the matcher to the doc
    matches = matcher(doc)
    # Create a Span for each match and assign the label 'LOCATION'
    # Overwrite the doc.ents with the matched spans
    doc.ents = [Span(doc, start, end, label="LOCATION") for match_id, start, end in matches]
    return doc

# Add the component to the pipeline after the 'ner' component
nlp.add_pipe(location_component, before='ner')

In [3]:
# Uses spacy to look up for location in a strings. Returns Array of matches
def nlpLocation(string):
    spacy_mathc = []
    for ent in nlp(string).ents:
        if ent.label_ == "GPE" or ent.label_ == "LOCATION":
            spacy_mathc.append(ent.text.lower())
    return spacy_mathc

def matchLocationV3(item_original):
    item = item_original.copy()
    locations = []

    locations += nlpLocation(item[2].replace("'","")) # Title
    locations += nlpLocation(item[3].replace("'","")) # Tags
    locations += nlpLocation(item[4].replace("'","")) # Descr
      
    item.append(locations)
    return item

# matchLocationV3(data2[405])

In [4]:
def removeDuplicateLocation(item_original):
    item = item_original.copy()
    item[7] = list(set(item[7]))
    return item

In [5]:
def printItem(item_original):
    print(item_original)
    print()
    return item_original

In [6]:
def onlyItemsWithPayload(item_original):
    if item_original[2] or item_original[3] or item_original[4]:
        return True
    return False

In [7]:
def onlyItemsWithMatchedLocation(item_original):
    if len(item_original[7]) > 0:
        return True
    return False

In [9]:
def countStats(acc, item):
    # Counting stats
    acc["total"] += 1
    if item[2] or item[3] or item[4]:
        acc["hasDataToAnalyze"] += 1
    if len(item[7]) > 0:
        acc["identified"] += 1
    return acc
def printStats(stats):
    print(stats["total"], "items were processed in total.")
    print(stats["hasDataToAnalyze"], "of them had title,description or tags to analyze.")
    print(stats["identified"], "out of", stats["hasDataToAnalyze"], "were matched with potential location")

In [16]:
from customPipeline import Pipe
data = data[:1800]

pl = Pipe()
# pl.addDataPipe(onlyItemsWithPayload)
pl.addItemPipe(matchLocationV3)
pl.addItemPipe(removeDuplicateLocation)
pl.addStatPipe(countStats, printStats, {"total":0, "hasDataToAnalyze":0, "identified":0})
pl.addDataPipe(onlyItemsWithMatchedLocation)
# pl.addPipe(printItem)
result = pl(data)

countStats: 100%|██████████| 1800/1800 [00:00<00:00, 1061849.11it/s]


1800 items were processed in total.
394 of them had title,description or tags to analyze.
19 out of 394 were matched with potential location


onlyItemsWithMatchedLocation: 100%|██████████| 1800/1800 [00:00<00:00, 1351306.10it/s]


In [15]:
len(result)

19